<a href="https://colab.research.google.com/github/ilgindogan/VDF_Prompting_Module2/blob/main/01_Shot_Strategies_Zero_One_Few.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 1 — Zero-shot / One-shot / Few-shot (Hands-on)






## Setup

Google Gemini or OpenAI key, definition




In [1]:
# Requirements
%pip -q install -U langchain-core langchain-openai langchain-google-genai tiktoken python-dotenv matplotlib pandas==2.2.2 pydantic==2.12.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.5 MB/s eta 0:00:00


In [25]:
import os, json, re
from typing import Dict, Any
from langchain_openai import AzureChatOpenAI

# --- Azure OpenAI Configuration ---
AZURE_ENDPOINT = "https://sd-rg.cognitiveservices.azure.com/"
AZURE_API_KEY = ""  # api key
AZURE_DEPLOYMENT = "vodafone_rag_module"
API_VERSION = "2024-12-01-preview"

def get_llm():
    """
    Initializes the Azure OpenAI client using the provided configuration.

    Returns:
        tuple: (llm_instance, provider_name_string)

    Raises:
        RuntimeError: If API credentials are missing or connection fails.
    """
    # Check if essential credentials are present
    if AZURE_API_KEY and AZURE_ENDPOINT:
        try:
            llm = AzureChatOpenAI(
                azure_deployment=AZURE_DEPLOYMENT,
                api_version=API_VERSION,
                azure_endpoint=AZURE_ENDPOINT,
                api_key=AZURE_API_KEY,
                temperature=0.1,
                max_retries=2
            )
            return llm, f'Azure OpenAI ({AZURE_DEPLOYMENT})'
        except Exception as e:
            raise RuntimeError(f"Azure connection error: {e}")

    raise RuntimeError('Azure API credentials are missing!')

# --- Initialization ---
try:
    llm, provider = get_llm()
    print('✅ LLM ready:', provider)

    # Uncomment the line below to test the connection immediately
    # print("Test Response:", llm.invoke("Hello, are you active?").content)

except Exception as e:
    # If initialization fails, set llm to None to prevent subsequent NameErrors
    llm = None
    print(f"❌ Error occurred: {e}")

def llm_text(prompt: str) -> str:
    """
    Sends a prompt to the LLM and retrieves the text response.

    Args:
        prompt (str): The input string to send to the model.

    Returns:
        str: The clean text response from the model.
    """
    if llm is None:
        return "Error: LLM is not initialized."

    resp = llm.invoke(prompt)
    # Safely retrieve content whether it's an object or string
    return getattr(resp, 'content', str(resp)).strip()

def strip_fences(s: str) -> str:
    """
    Removes Markdown code fences (e.g., ```json ... ```) from a string.

    Args:
        s (str): The input string containing code fences.

    Returns:
        str: Cleaned string without the fences.
    """
    s = s.strip()
    # Remove starting ```json or ``` (case insensitive)
    s = re.sub(r'^```(json)?\s*', '', s, flags=re.IGNORECASE)
    # Remove ending ```
    s = re.sub(r'\s*```$', '', s)
    return s.strip()

✅ LLM hazır: Azure OpenAI (vodafone_rag_module)


In [26]:
# Dataset for email triage
EMAILS = [
    {'id': 'E1', 'text': 'Kargom hâlâ gelmedi. 7 gündür bekliyorum. Acil çözüm istiyorum!', 'notes': 'Gecikme + yüksek aciliyet'},
    {'id': 'E2', 'text': 'Ürün kırık geldi. Değişim yapabilir miyiz?', 'notes': 'Hasarlı ürün'},
    {'id': 'E3', 'text': 'İade sürecini nasıl başlatabilirim? Kutuyu attım ama ürün duruyor.', 'notes': 'İade + edge-case (kutusuz)'},
    {'id': 'E4', 'text': 'Kartımdan iki kez çekim yapılmış görünüyor. Lütfen hemen kontrol edin.', 'notes': 'Faturalama + yüksek aciliyet'},
    {'id': 'E5', 'text': 'Ürününüzün kullanım kılavuzunu paylaşır mısınız?', 'notes': 'Bilgi talebi (low)'},
]
len(EMAILS)

5

## 1) Görev: Email triage → JSON

Çıktıyı bir **output contract** ile kilitliyoruz:
- `category`: sınıf adı
- `urgency`: low | medium | high
- `reason`: **tek cümle** gerekçe

Amaç: üretimde parse edilebilir, stabil çıktı.

In [27]:
SCHEMA = (
    'Return ONLY valid JSON with exactly these keys:\n'
    '{\n'
    '  "category": "string",\n'
    '  "urgency": "low|medium|high",\n'
    '  "reason": "string (max 1 sentence)"\n'
    '}\n'
    'No extra text, no markdown, JSON only.'
)

import json

## 2) Prompt fonksiyonları

### Zero-shot
Örnek yok. Instruction + contract + input.

### One-shot
Tek örnekle output kalıbını ankore eder.

### Few-shot
3 örnekle pattern genellemesi + edge-case güçlenir.

In [28]:
def prompt_zero_shot(email_text: str) -> str:
    return (
        'You are a customer support triage assistant.\n'
        'Classify the email into a category and urgency.\n\n'
        + SCHEMA + '\n\n'
        + 'Email:\n' + email_text
    )

ONE_EXAMPLE = {
    'email': 'My order arrived broken. I want a replacement as soon as possible.',
    'answer': {
        'category': 'Damaged product',
        'urgency': 'medium',
        'reason': 'Customer reports a damaged item and requests a replacement.'
    }
}

def prompt_one_shot(email_text: str) -> str:
    return (
        'You are a customer support triage assistant.\n'
        'Use the example to follow the same output format and labeling style.\n\n'
        + SCHEMA + '\n\n'
        + 'Example:\nEmail:\n' + ONE_EXAMPLE['email'] + '\n'
        + 'Answer:\n' + json.dumps(ONE_EXAMPLE['answer'], ensure_ascii=False) + '\n\n'
        + 'Now classify this email:\n\nEmail:\n' + email_text
    )

FEW_EXAMPLES = [
    (
        'Where is my package? It was supposed to arrive 5 days ago.',
        {'category':'Delivery issue','urgency':'high','reason':'Customer reports a significantly delayed delivery.'}
    ),
    (
        'How can I return the product? I changed my mind.',
        {'category':'Return request','urgency':'low','reason':'Customer asks for return instructions without a critical issue.'}
    ),
    (
        'You charged me twice for the same order. Fix this immediately.',
        {'category':'Billing issue','urgency':'high','reason':'Customer reports a double charge and requests urgent resolution.'}
    ),
]

def prompt_few_shot(email_text: str) -> str:
    ex_block = ''
    for mail, ans in FEW_EXAMPLES:
        ex_block += 'Email:\n' + mail + '\nAnswer:\n' + json.dumps(ans, ensure_ascii=False) + '\n\n'
    return (
        'You are a customer support triage assistant.\n'
        'Follow the same pattern as the examples.\n\n'
        + SCHEMA + '\n\n'
        + 'Examples:\n' + ex_block
        + 'Now classify this email:\n\nEmail:\n' + email_text
    )

## 3) Run + Parse

Çıktıyı JSON parse ediyoruz. Eğer model formatı bozarsa hata yakalanacak.

In [29]:
def run_and_parse(prompt: str) -> Dict[str, Any]:
    raw = llm_text(prompt)
    raw2 = strip_fences(raw)
    try:
        return json.loads(raw2)
    except json.JSONDecodeError:
        print('⚠️ JSON parse edilemedi. Ham çıktı:\n', raw)
        raise

def run_strategy(name: str, make_prompt, email_text: str) -> Dict[str, Any]:
    print('\n===', name, '===')
    out = run_and_parse(make_prompt(email_text))
    print(json.dumps(out, ensure_ascii=False, indent=2))
    return out

## 4) Tek email üzerinde karşılaştırma

In [30]:
test_email = EMAILS[0]['text']
print('TEST:', test_email)
_ = run_strategy('ZERO-SHOT', prompt_zero_shot, test_email)
_ = run_strategy('ONE-SHOT', prompt_one_shot, test_email)
_ = run_strategy('FEW-SHOT', prompt_few_shot, test_email)

TEST: Kargom hâlâ gelmedi. 7 gündür bekliyorum. Acil çözüm istiyorum!

=== ZERO-SHOT ===
{
  "category": "Shipping Delay",
  "urgency": "high",
  "reason": "Customer has been waiting 7 days for their shipment and requests urgent resolution."
}

=== ONE-SHOT ===
{
  "category": "Delayed delivery",
  "urgency": "high",
  "reason": "Customer has been waiting 7 days for their shipment and requests urgent resolution."
}

=== FEW-SHOT ===
{
  "category": "Delivery issue",
  "urgency": "high",
  "reason": "Customer reports a 7-day delayed delivery and requests urgent resolution."
}


## 5) Batch karşılaştırma (5 email)

Sonuçları tablo gibi görüp tutarlılığı tartışın.

In [31]:
import pandas as pd

def batch_run(make_prompt):
    rows = []
    for e in EMAILS:
        out = run_and_parse(make_prompt(e['text']))
        rows.append({
            'id': e['id'],
            'notes': e['notes'],
            'category': out.get('category'),
            'urgency': out.get('urgency'),
            'reason': out.get('reason'),
        })
    return rows

df_zero = pd.DataFrame(batch_run(prompt_zero_shot)).add_prefix('zero_')
df_one  = pd.DataFrame(batch_run(prompt_one_shot)).add_prefix('one_')
df_few  = pd.DataFrame(batch_run(prompt_few_shot)).add_prefix('few_')

df = pd.concat([
    df_zero[['zero_id','zero_notes','zero_category','zero_urgency']],
    df_one[['one_category','one_urgency']],
    df_few[['few_category','few_urgency']],
], axis=1)
df

,zero_id,zero_notes,zero_category,zero_urgency,one_category,one_urgency,few_category,few_urgency
0,E1,Gecikme + yüksek aciliyet,Shipping Delay,high,Late delivery,high,Delivery issue,high
1,E2,Hasarlı ürün,Product Issue,high,Damaged product,medium,Product issue,medium
2,E3,İade + edge-case (kutusuz),Return Process,medium,Return process,medium,Return request,medium
3,E4,Faturalama + yüksek aciliyet,Billing Issue,high,Billing issue,high,Billing issue,high
4,E5,Bilgi talebi (low),Product Information,low,Product information request,low,Product information,low


## 6) Egzersiz (3–5 dk)

1) `FEW_EXAMPLES` içine **edge-case** bir örnek ekleyin.
2) `category` adlarını sabitleyin (Delivery issue, Billing issue, ...).
3) Aynı email'i 3 kez çalıştırıp tutarlılığı inceleyin.

